# All American Transfers

In [ ]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage
from recordlinkage.index import Full
import queries

## Build out DataFrames
> You still need to run this whole notebook for either ranked or unranked - that still needs to be cleaned up.

In [ ]:
ranked = False
query = queries.get_query_Transfers247(ranked)
df_247 = (fx.connDBAndReturnDF(query)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

In [ ]:
query = queries.get_query_UnlinkedAllAmerican()
df_aa = (fx.connDBAndReturnDF(query)).set_index('ID')
df_aa.index.name = 'AllAmerican_ID'
df_aa

### NOTES

> you don't

In [ ]:
indexer = recordlinkage.Index()
indexer.add(Full())

In [ ]:
pairs = indexer.index(df_aa, df_247)

In [ ]:
from functions import YearOther

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
#c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
#sumFields.append('StandardizedPosition')
#c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
#sumFields.append('KeyPositionGroup')
c.add(YearOther('Year', 'Year', label='Year'))
sumFields.append('Year')

In [ ]:
features = c.compute(pairs, df_aa, df_247)

In [ ]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['Sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

In [ ]:
features.insert(0, 'sourceID', features.index.get_level_values(0))
features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['Sum'].idxmax()]
    #.94
    if (data['Sum'] >.95):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsAllAmericanTransfer.csv")

In [ ]:
dfFinal

In [ ]:
fuzzyMI = pd.MultiIndex.from_frame(dfFinal)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_AATransfers.json",
    fuzzyMI[0:200],
    df_aa,
    df_247,
    dataset_a_name="All American",
    dataset_b_name="Master"
)

In [ ]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result-AAUnranked.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

In [ ]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 6, 1, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence, Transfer)
        VALUES (?,?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()